In [1]:
from Systems import UnbalancedDisc
from Systems import NoisyUnbalancedDisc
from Systems import DiscreteUnbalancedDisc
import deepSI
from matplotlib import pyplot as plt
import numpy as np
#import autograd
import torch
import time
from torch import nn
#import quadprog as qp
import qpsolvers as qp

In [2]:
system = NoisyUnbalancedDisc(dt=0.1, sigma_n=[0, 0])

class I_encoder(deepSI.fit_systems.SS_encoder):
    def __init__(self, nx = 2, na=2, nb=2, feedthrough=False) -> None:
        super().__init__(nx=nx, na=na, nb=nb, feedthrough=feedthrough)

    def init_nets(self, nu, ny): # a bit weird
        ny = ny if ny is not None else 1
        nu = nu if nu is not None else 1
        self.encoder = self.e_net(self.nb*nu+self.na*ny, self.nx, n_nodes_per_layer=self.e_n_nodes_per_layer, n_hidden_layers=self.e_n_hidden_layers, activation=self.e_activation)
        self.fn =      self.f_net(self.nx+nu,            self.nx, n_nodes_per_layer=self.f_n_nodes_per_layer, n_hidden_layers=self.f_n_hidden_layers, activation=self.f_activation)
        hn_in = self.nx + nu if self.feedthrough else self.nx
        self.hn =      nn.Identity(hn_in)#

I_enc = deepSI.load_system("systems/UnbalancedDisk_dt01_e100_SNR_100")

In [3]:
# function that converts torch nn to casadi expression
from casadi import *

def CasADiHn(ss_enc, x):
    n_hidden_layers = ss_enc.h_n_hidden_layers

    params = {}
    for name, param in ss_enc.hn.named_parameters():
        params[name] = param.detach().numpy()
    params_list = list(params.values())

    temp_nn = x
    for i in range(n_hidden_layers):
        W_NL = params_list[2+i*2]
        b_NL = params_list[3+i*2]
        temp_nn = mtimes(W_NL, temp_nn)+b_NL
        temp_nn = tanh(temp_nn)
    W_NL = params_list[2+n_hidden_layers*2]
    b_NL = params_list[3+n_hidden_layers*2]
    nn_NL = mtimes(W_NL, temp_nn)+b_NL

    W_Lin = params_list[0]
    b_Lin = params_list[1]
    nn_Lin = mtimes(W_Lin,x) + b_Lin

    return nn_NL + nn_Lin

def CasADiFn(ss_enc, x, u):
    n_hidden_layers = ss_enc.f_n_hidden_layers

    params = {}
    for name, param in ss_enc.fn.named_parameters():
        params[name] = param.detach().numpy()
    params_list = list(params.values())
    
    xu = vertcat(x,u)

    temp_nn = xu
    for i in range(n_hidden_layers):
        W_NL = params_list[2+i*2]
        b_NL = params_list[3+i*2]
        temp_nn = mtimes(W_NL, temp_nn)+b_NL
        temp_nn = tanh(temp_nn)
    W_NL = params_list[2+n_hidden_layers*2]
    b_NL = params_list[3+n_hidden_layers*2]
    nn_NL = mtimes(W_NL, temp_nn)+b_NL

    W_Lin = params_list[0]
    b_Lin = params_list[1]
    nn_Lin = mtimes(W_Lin,xu) + b_Lin

    return nn_NL + nn_Lin

In [4]:
# declared sym variables
x = MX.sym("x",I_enc.nx,1)
nu = I_enc.nu if I_enc.nu is not None else 1
u = MX.sym("u",nu,1)

# convert torch nn to casadi function
rhs = CasADiFn(I_enc, x, u)
f = Function('f', [x, u], [rhs])

#y_rhs = CasADiHn(I_enc, x)
#h = Function('h', [x], [y_rhs])

# apply correction to casadi function such that fc(0) = 0
correction_f = f([0,0], 0)
rhs_c = rhs - correction_f
#correction_h = h([0,0])
#y_rhs_c = y_rhs - correction_h
f_c = Function('f_c', [x, u], [rhs_c])
#h_c = Function('h_c', [x], [y_rhs_c])

In [5]:
# Box constraints
x_min = [-10, -10]
x_max = [10, 10]
u_min = -4
u_max = 4

# Initial and final values
x0 = [0,0]
x_ref = [0.001, 1.0]
u_ref = 0

# Weight matrices for the cost function
Q = np.array([[1,0],[0,100]])
R = 1

# MPC parameters
dt = 0.1
Nc = 4
Nsim = 50
#dlam = 0.01
stages = 100

In [6]:
L = np.zeros([3*stages,1])
lam = 0
dlam = 1.0/stages

for i in range(stages):
    L[i] = lam
    L[i+stages] = lam + dlam/2
    L[i+2*stages] = lam + dlam

    lam = lam + dlam

n_states = np.shape(x)[0]
nx = n_states
n_controls = np.shape(u)[0]
nu = n_controls

In [7]:
Jfx = Function("Jfx", [x, u], [jacobian(rhs_c,x)])
Jfu = Function("Jfu", [x, u], [jacobian(rhs_c,u)])

In [8]:
def par_lpv_int(x,nx,u,nu,Jfx,Jfu,dlam,stages,L):
#     # FUNCTION LPV_INT
#     # RK4 integrator with chosen resolution and internal stages
#     # used to get A,B matrices symbolically to be used at gridpoints
    
#     A = np.zeros([nx,nx])
#     B = np.zeros([nx,nu])

#     Lx = np.kron(L,x)
#     #Lx = reshape(kron(L,x), (2,stages*3)).T
#     Lu = L*u

#     Fx = Jfx.map(stages)
#     kx = Fx(Lx.T, Lu.T)
#     Fu = Jfu.map(stages)
#     ku = Fu(Lx.T, Lu.T)

#     A = np.zeros((2,2))
#     A[:,0] = reshape(sum2(kx[:,0:stages*2:2]), (1,2)) # k1
#     A[:,1] = reshape(sum2(kx[:,1:stages*2:2]), (1,2)) # k1
#     A[:,0] = A[:,0] + 4*reshape(sum2(kx[:,stages*2:stages*4:2]), (1,2)) # k2
#     A[:,1] = A[:,1] + 4*reshape(sum2(kx[:,stages*2+1:stages*4:2]), (1,2)) # k2
#     A[:,0] = A[:,0] + reshape(sum2(kx[:,stages*4:stages*6:2]), (1,2)) # k4
#     A[:,1] = A[:,1] + reshape(sum2(kx[:,stages*4+1:stages*6:2]), (1,2)) # k4
#     A = 1/6*dlam*A

#     B = np.zeros((2,1))
#     B[:,0] = reshape(sum2(ku[:,0:stages:1]), (1,2)) # k1
#     B[:,0] = B[:,0] + 4*reshape(sum2(ku[:,stages:2*stages:1]), (1,2)) # k2
#     B[:,0] = B[:,0] + reshape(sum2(ku[:,2*stages:3*stages:1]), (1,2)) # k4
#     B = 1/6*dlam*B
            
    return 1

In [9]:
def lpv_int(x,nx,u,nu,Jfx,Jfu,dlam,stages):
    # FUNCTION LPV_INT
    # RK4 integrator with chosen resolution and internal stages
    # used to get A,B matrices symbolically to be used at gridpoints
    
    A = np.zeros([nx,nx])
    B = np.zeros([nx,nu])
    
    lam = 0

    for i in np.arange(stages):
        k1 = Jfx(lam*x,lam*u)
        j1 = Jfu(lam*x,lam*u)

        k2 = Jfx((lam+dlam/2)*x,(lam+dlam/2)*u)
        j2 = Jfu((lam+dlam/2)*x,(lam+dlam/2)*u)

        k4 = Jfx((lam+dlam)*x,(lam+dlam)*u)
        j4 = Jfu((lam+dlam)*x,(lam+dlam)*u)

        A = A + 1/6*dlam*(k1 + 4*k2 + k4)
        B = B + 1/6*dlam*(j1 + 4*j2 + j4)
        lam = lam + dlam
            
    return A,B

In [10]:
[A, B] = lpv_int(x,2,u,1,Jfx,Jfu,dlam,stages)
get_A = Function("get_A",[x,u],[A])
get_B = Function("get_B",[x,u],[B])

In [11]:
# xt = np.tile(x0, Nc)
# ut = np.zeros(Nc*nu)
Get_A = get_A.map(Nc, "thread", 32)
Get_B = get_B.map(Nc, "thread", 32)

In [12]:
import itertools

def getPhi(list_A, Nc, nx, nu):
    Phi = np.zeros([nx*Nc, nx])
    for i in range(Nc):
        temp = np.eye(nx)
        for j in range(i,-1,-1):
            temp = np.matmul(temp, list_A[(nx*j):(nx*j+nx),:])
        Phi[i*nx:(i+1)*nx, :] = temp
    return Phi

def getGamma(list_A, list_B, Nc, nx, nu):
    Gamma = np.zeros([nx*Nc, nu*Nc])
    for i in range(Nc):
        for j in range(0,i+1):
            temp = np.eye(nx)
            for l in range(i-j,-1,-1):
                if l == 0:
                    temp = np.matmul(temp, list_B[(nx*j):(nx*j+nx),:])
                else:
                    temp = np.matmul(temp, list_A[(nx*l):(nx*l+nx),:])
            Gamma[i*nx:nx*(i+1),j*nu:(j+1)*nu] = temp
    return Gamma

def getPsi(Nc, R):
    return np.kron(np.eye(Nc), R)

def getOmega(Nc, Q):
    return np.kron(np.eye(Nc), Q)

def getDEMc(x_min, x_max, u_min, u_max, Nc, nx, nu):
    bi = np.array([list(itertools.chain([-u_min, u_max], [x*-1 for x in x_min],  x_max))])
    bN = np.array([list(itertools.chain([x*-1 for x in x_min],  x_max))])
    c = np.hstack((np.tile(bi, Nc), bN)).T

    In = np.eye(nx)
    Im = np.eye(nu)
    Zn = np.zeros((nu,nx))
    Zm = np.zeros((nx,nu))
    
    Mi = np.vstack((Zn, Zn, -In, In))
    Mn = np.vstack((-In, In))
    M = (np.zeros((Nc*2*(nx+nu)+2*nx, Nc*nx)))
    M[Nc*2*(nx+nu):,(Nc-1)*nx:] = Mn
    M[2*(nx+nu):Nc*2*(nx+nu),:(Nc-1)*nx] = np.kron(np.eye(Nc-1), Mi)

    Ei = np.vstack((-Im, Im, Zm, Zm))
    E = np.vstack((np.kron(np.eye(Nc), Ei), np.zeros((nx*2, Nc*nu))))

    D = np.zeros((Nc*2*(nx+nu)+2*nx, nx))
    D[:2*(nx+nu),:] = Mi

    return D, E, M, c

In [13]:
# list_A = np.ones([Nc*nx, nx])
# list_B = np.ones([Nc*nx, nu])
# Phi = getPhi(list_A, Nc, nx, nu)
# Gamma = getGamma(list_A, list_B, Nc, nx, nu)

# D, E, M, B = getDEMc(x_min, x_max, u_min, u_max, Nc, nx, nu)
# L = (M@Gamma) + E
# W = -D - (M@Phi)
# W.shape, L.shape

In [37]:
# logging list
x0 = np.array([0.0,0.5])
u_log = np.zeros(Nsim*n_controls)
x_log = np.zeros((Nsim+1)*n_states)
x_log[:nx] = x0
components_times = np.zeros(3) # getAB, solve, denorm and sim
component_start = 0
t = np.zeros(Nsim)
t0 = 0
comp_t_log = np.zeros(Nsim)
start = time.time()
lpv_counter = np.zeros(Nsim,int)

# normalize reference list
norm = I_enc.norm
#reference_list_normalized = (reference_list - norm.y0[1])/norm.ystd[1]
x0_norm = (x0 - norm.y0)/norm.ystd

# set initial values for x
x = np.tile(x0_norm, Nc)
u = np.zeros(Nc*nu)

list_A = np.zeros([Nc*nx, nx])
list_B = np.zeros([Nc*nx, nu])
list_A_p = np.zeros([Nc*nx, nx])
list_B_p = np.zeros([Nc*nx, nu])
# pA = np.zeros([Nc*nx, nx])
# pB = np.zeros([Nc*nx, nu])
Psi = getPsi(Nc, R)
Omega = getPsi(Nc, Q)
D, E, M, c = getDEMc(x_min, x_max, u_min, u_max, Nc, nx, nu)

ne = 1
Ge = np.zeros((Nc+ne, Nc+ne))

for mpciter in range(1):
    start_time_iter = time.time()
    
    while True:
        # for i in np.arange(Nc):
        #     list_A[(n_states*i):(n_states*i+n_states),:] = get_A(x[i*nx:(i+1)*nx],u[i*nu:(i+1)*nu])
        #     list_B[(n_states*i):(n_states*i+n_states),:] = get_B(x[i*nx:(i+1)*nx],u[i*nu:(i+1)*nu])
        
        pA = Get_A(np.vstack(np.split(x,Nc)).T,u)
        for i in range(Nc):
            list_A[(n_states*i):(n_states*i+n_states),:] = pA[:,i*nx:(i+1)*nx]
        pB = Get_B(np.vstack(np.split(x,Nc)).T,u)
        for i in range(Nc):
            list_B[(n_states*i):(n_states*i+n_states),:] = pB[:,i*nu:(i+1)*nu]

        Phi = getPhi(list_A, Nc, nx, nu)
        Gamma = getGamma(list_A, list_B, Nc, nx, nu)
        G = 2*(Psi+(Gamma.T@Omega@Gamma))
        F = 2*(Gamma.T@Omega@Phi)
        L = (M@Gamma) + E
        W = -D - (M@Phi)

        Le = np.hstack((L, -np.ones((Nc*2*(nx+nu)+2*nx,1))))
        Ge[:Nc, :Nc] = G
        Ge[Nc:,Nc:] = 100
        Fe = np.hstack((F@x[:2], np.zeros(ne)))
        

        u_old = u
        #x_ss = x[:2] - [0, 1]
        #u = -np.linalg.inv(G)@F@x[:2]
        #u = qp.solve_qp(G,F@x[:2],L,(W@x[:2]) + c[:,0], solver="quadprog")
        ue = qp.solve_qp(Ge,Fe,Le,(W@x[:2]) + c[:,0], solver="quadprog")
        u = ue[:Nc]
         
        x[nx:Nc*nx] = ((Phi@x[:2]) + Gamma@u)[:(Nc-1)*nx]# + np.tile(np.array(correction_f)[:,0], (Nc-1))
        
        lpv_counter[mpciter] += 1
        if (lpv_counter[mpciter] >= 5) or (np.linalg.norm(u-u_old) < 1e-7):
            break

    print("MPC iteration: ", mpciter+1)
    print("LPV counter: ", lpv_counter[mpciter])
    
    t[mpciter] = t0
    t0 = t0 + dt
    
    # denormalize x and u
    x_denormalized = norm.ystd*x0_norm + norm.y0
    u_denormalized = norm.ustd*u[0] + norm.u0

    # make system step and normalize
    x_denormalized = system.f(x_denormalized, u_denormalized)
    x_measured = system.h(x_denormalized, u_denormalized)
    x0_norm = (x_measured - norm.y0)/norm.ystd

    x_log[(mpciter+1)*nx:(mpciter+2)*nx] = x_measured
    u_log[mpciter] = u_denormalized
    
    x = np.hstack((x[nx:(Nc+1)*nx],x[-2:]))
    x[:nx] = x0_norm
    u = np.hstack((u[nx:Nc*nx],u[-2:]))

    # finished mpc time measurement
    end_time_iter = time.time()
    comp_t_log[mpciter] = end_time_iter - start_time_iter

end = time.time()
runtime = end - start
print(runtime)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 5 is different from 4)

In [36]:
Fe

array([ -32.43893716, -179.80506173, -121.91653969,  -27.53204945,
          0.        ])

In [ ]:
x_reference_list = np.load("references/randomLevelTime15_20Range-1_1Nsim500.npy")
x_reference_list_normalized = ((x_reference_list.T - norm.y0)/norm.ystd).T
y_reference_list = x_reference_list_normalized[1,:]
#plt.plot(y_reference_list)

In [ ]:
y_reference_list[0]

In [ ]:
def getXsUs(y_reference_list_normalize, nx, nu, ny, Nsim, u_min, u_max, x_min, x_max, get_A, get_B, C, f0, h0):
    ne = 1 #number of variables in epsilon
    Q = np.eye(ny) # add this as variable of function
    R = np.eye(nu) # add this as variable of function
    lam = 100
    
    In = np.eye(nx)
    Im = np.eye(nu)
    Zn = np.zeros((nu,nx))
    Zm = np.zeros((nx,nu))

    Mi = np.vstack((Zn, Zn, -In, In))
    Ei = np.vstack((-Im, Im, Zm, Zm))
    h = np.array([list(itertools.chain([-u_min, u_max], [x*-1 for x in x_min],  x_max))]).T

    T = np.zeros((2*(nx+nu), nx+nu+ne))
    T[:,:nx] = Mi
    T[:,nx:nx+nu] = Ei
    T[:,nx+nu:] = -np.ones((2*(nx+nu),ne))

    b = np.zeros((nx+ny, 1))
    b[:nx] = f0

    P = np.zeros((nx+nu+ne, nx+nu+ne))
    P[:nx, :nx] = C.T@Q@C
    P[nx:nx+nu, nx:nx+nu] = R
    P[nx+nu:, nx+nu:] = lam

    q = np.zeros((nx+nu+ne,1))
    
    xs = np.zeros(nx)
    us = np.zeros(nu)
    xue = np.zeros(nx+nu+ne)
    As = np.zeros((nx,nx))
    Bs = np.zeros((nx,nu))
    A = np.zeros((nx+ny, nx+nu+ne))
    A[nx:nx+ny,:nx] = C #change this to getC from xs us when needed

    x_reference_list_normalized = np.zeros((nx, Nsim))
    u_reference_list_normalized = np.zeros((nu, Nsim))
    e_reference_list_normalized = np.zeros((ne, Nsim))

    for j in range(Nsim):

        b[nx:nx+ny] = y_reference_list_normalize[j] - h0 #+ correction_h #add h0 here when needed
        q[:nx,0] = C.T@Q@(h0 - y_reference_list_normalize[j])

        for i in range(10):
            As[:,:] = get_A(xs, us)
            Bs[:,:] = get_B(xs, us)

            A[:nx,:nx] = np.eye(nx) - As
            A[:nx,nx:nx+nu] = -Bs
            #A[nx:,:nx] = C
            #q[:nx,0] = C.T@Q@h0 - C.T@Q@

            #xu[:] = (np.linalg.inv(A)@b)[:,0]
            xue[:] = (qp.solve_qp(P,q,T,h[:,0],A,b[:,0],solver="osqp"))

            xold = xs
            uold = us
            xs = xue[:nx]
            us = xue[nx:nx+nu]
            e = xue[nx+nu:]

            if np.linalg.norm(xs-xold) <= 1e-5 and np.linalg.norm(us-uold) <= 1e-5:
                break

        x_reference_list_normalized[:,j] = xs
        u_reference_list_normalized[:,j] = us
        e_reference_list_normalized[:,j] = e
        
    return x_reference_list_normalized, u_reference_list_normalized, e_reference_list_normalized

In [ ]:
C = np.array([[0, 1]])
ny = 1
h0 = np.zeros(1)
Nsim = 100
x_reference_list_normalized, u_reference_list_normalized, e_reference_list_normalized = getXsUs(y_reference_list, nx, nu, ny, Nsim, \
    u_min, u_max, x_min, x_max, get_A, get_B, C, correction_f, h0)

In [ ]:
#x_reference_list_normalized[:,14:25]#, u_reference_list_normalized[:,:4]
plt.plot(x_reference_list_normalized[1,:])
#x_reference_list_normalized

In [ ]:
plt.plot(e_reference_list_normalized[0,:])

In [ ]:
#x_complete = np.hstack((x_log))

plt.subplot(2,3,1)
plt.plot(x_log[0::nx])

plt.subplot(2,3,2)
plt.plot(x_log[1::nx])

plt.subplot(2,3,3)
plt.plot(u[:])

plt.subplot(2,3,4)
plt.plot(comp_t_log[:])